In [1]:
#! /usr/bin/python3
from Crypto.Cipher import AES
import chipwhisperer as cw
from common import cw_connect, cw_set_params, reset_arm_target, use_fpga_pll
import time
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

pscope, ftarget = cw_connect()
target, fpga_io, nSamples = cw_set_params(pscope, ftarget)
use_fpga_pll(ftarget)

Working with:
UnitInfo(driver=<picosdk.ps5000a.Ps5000alib object at 0x7f1d1881cd60>, variant=b'5244D', serial=b'KU687/0175')
/dev/ttyS4 - n/a
Found the following picoscope:
DriverVersion                 : PS5000A Linux Driver, 2.1.124.5349
USBVersion                    : 3.0
HardwareVersion               : 1
VariantInfo                   : 5244D
BatchAndSerial                : KU687/0175
CalDate                       : 26Apr23
KernelVersion                 : 0.0
DigitalHardwareVersion        : 1
AnalogueHardwareVersion       : 1
PicoFirmwareVersion1          : 1.7.15.0
PicoFirmwareVersion2          : 1.2.34.0
Nsamples :  1612
Sampling interval = 3.224000 us


In [2]:

def reset_flush():
    reset_arm_target(ftarget)
    target.flush()

def pico_capture(ps, nSamples):
    # Arm the picoscope
    ps.runBlock()
    time.sleep(0.05)
    # Trigger the encryption on Target
    target.fpga_write(target.REG_USER_LED, [0x01])
    target.usb_trigger_toggle()
    ps.waitReady()
    # Capture the trace 
    data = ps.getDataV('A', nSamples, returnOverflow=False)
    return data

In [4]:
# Reset before using
reset_flush()

ktp = cw.ktp.Basic()
# Initialize cipher to verify DUT result:
key, plaintext = ktp.next()
cipher = AES.new(bytes(key), AES.MODE_ECB)
print("Key: ", [hex(el) for el in key])
print("Plaintext: ", [hex(el) for el in plaintext])

output_len = 16
# Dummy capture call due to bug of using AC coupling
pico_capture(pscope, nSamples)

target.simpleserial_write('k', key[::-1])
target.simpleserial_write('p', plaintext[::-1])
wave = pico_capture(pscope, nSamples)

response = target.simpleserial_read('r', output_len, ack=True)
print("Ciphertext: ", [hex(el) for el in response])
print("Expected : ", [hex(el) for el in cipher.encrypt(bytes(plaintext))])

output_notebook()
p = figure(plot_width=800)

xrange = range(len(wave))
p.line(xrange, wave, line_color="red")
show(p)


assert (list(response) == list(cipher.encrypt(bytes(plaintext)))), "Incorrect encryption result!\nGot {}\nExp {}\n".format(list(response), list(plaintext))
    

Key:  ['0x2b', '0x7e', '0x15', '0x16', '0x28', '0xae', '0xd2', '0xa6', '0xab', '0xf7', '0x15', '0x88', '0x9', '0xcf', '0x4f', '0x3c']
Plaintext:  ['0x11', '0xb6', '0x44', '0xfc', '0xcc', '0x60', '0xa7', '0x17', '0xa5', '0x9f', '0xb', '0x54', '0xfd', '0x41', '0xbe', '0xac']
Ciphertext:  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']
Expected :  ['0x28', '0x96', '0x71', '0x25', '0xaf', '0x23', '0x78', '0xf5', '0xac', '0xf5', '0x97', '0x57', '0x81', '0xdb', '0xa4', '0x98']


Loading BokehJS ...

AssertionError: Incorrect encryption result!
Got [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Exp [17, 182, 68, 252, 204, 96, 167, 23, 165, 159, 11, 84, 253, 65, 190, 172]


In [ ]:
# Disconnect for all devices
target.dis()